# Cross-Entropy (softamx) : $L_i = - log( \frac{exp(s_{y_i})}{\sum_j exp(s_{j})} )$ 

## 1. **损失函数**

我们从 交叉熵损失 开始。对于一个样本 $i$，损失函数 $L_i$ 是：

$$
L_i = - \log(p_{y[i]})
$$

其中 $p_{y[i]}$ 是样本 $i$ 属于正确类别 $y[i]$ 的概率。

$$
p_j = \frac{e^{s_j}}{\sum_k e^{s_k}}
$$

是 softmax 输出的类别 $j$ 的概率，其中 $s_j$ 是模型对类别 $j$ 的得分（即 $s_j = x_i^T W_j$）。

## 2. **求梯度**

我们需要求损失函数 $L_i$ 对 得分 $s_j$ 的梯度。使用链式法则来推导：

$$
\frac{\partial L_i}{\partial s_j} = \frac{\partial L_i}{\partial p_j} \cdot \frac{\partial p_j}{\partial s_j}
$$

**步骤 1**：求交叉熵损失对 $p_j$ 的梯度

交叉熵损失是对 $p_{y[i]}$（正确类别概率）取对数的负数，所以：

$$
\frac{\partial L_i}{\partial p_j} = 
\begin{cases}
- \frac{1}{p_{y[i]}} & \text{if } j = y[i] \\
\frac{1}{p_j} & \text{if } j \neq y[i]
\end{cases}
$$

对于正确类别 $j = y[i]$，梯度为 $-\frac{1}{p_{y[i]}}$，这是因为我们希望最大化正确类别的概率，从而最小化损失。对数是单调递减的，所以当正确类别的概率增大时，损失减少。

对于其他类别 $j \neq y[i]$，梯度为 $\frac{1}{p_j}$，表示减少错误类别的概率。

**步骤 2**：求 softmax 函数对得分 $s_j$ 的梯度

softmax 输出 $p_j$ 关于得分 $s_j$ 的梯度为：

$$
\frac{\partial p_j}{\partial s_j} =
\begin{cases}
p_j (1 - p_j) & \text{if } j = y[i] \\
- p_j p_{y[i]} & \text{if } j \neq y[i]
\end{cases}
$$

**步骤 3**：结合链式法则

将交叉熵损失对 $p_j$ 的梯度和 softmax 对 $s_j$ 的梯度结合起来，得到损失对 $s_j$ 的梯度：

$$
\frac{\partial L_i}{\partial s_j} = \frac{\partial L_i}{\partial p_j} \cdot \frac{\partial p_j}{\partial s_j}
$$

对于正确类别 $j = y[i]$：

$$
\frac{\partial L_i}{\partial s_{y[i]}} = \left( - \frac{1}{p_{y[i]}} \right) \cdot p_{y[i]} (1 - p_{y[i]}) = p_{y[i]} - 1
$$

对于其他类别 $j \neq y[i]$：

$$
\frac{\partial L_i}{\partial s_j} = \left( \frac{1}{p_j} \right) \cdot \left( - p_j p_{y[i]} \right) = p_j
$$

## 3. **为什么 $j = y[i]$ 时梯度是负的？**

这是因为我们希望增加正确类别的概率，而损失函数是关于正确类别概率 $p_{y[i]}$ 的负对数。负对数函数是单调递减的，所以当 $p_{y[i]}$ 增加时，损失会减小。

损失函数的梯度是损失对 $p_{y[i]}$ 的变化率，因此，对于正确类别 $y[i]$，我们需要增加其概率，从而减少损失。这导致了梯度是负数，即 $(p_{y[i]} - 1)$，因为对数函数的性质使得我们需要通过减去 1 来促使 $p_{y[i]}$ 增加，从而减小损失。

## 4. **总结**

对于正确类别 $j = y[i]$，梯度是负的 $(p_{y[i]} - 1)$，因为我们希望增大正确类别的概率，从而减少损失。

对于其他类别 $j \neq y[i]$，梯度是正的，因为我们希望减小其他类别的概率，使得正确类别的概率相对增大。

这个负号是通过交叉熵损失函数的对数形式和 softmax 输出的性质推导出来的。希望这个详细推导能帮助你理解这个梯度的来源！


# SVM Loss with margin : $L_i = \sum_{j\neq y_i} max(0, s_j - s_{y_i} +1 )$

### **1. SVM 损失函数的定义**

在 **结构化 SVM** 中，我们使用的是 **大间隔分类** 的目标，损失函数是通过以下公式定义的：

$$
L_i = \sum_{j \neq y[i]} \max(0, s_j - s_{y[i]} + 1)
$$

其中：
- $s_j$ 是类别 $j$ 的得分，计算方式是： $s_j = W^T x_i$。
- $s_{y[i]}$ 是正确类别 $y[i]$ 的得分。

**目标**：确保正确类别的得分比其他类别的得分大 **至少** 1，同时在训练时 **最大化间隔**。

### **2. 损失函数对权重 $W$ 的梯度计算**

我们要计算的是损失函数 $L_i$ 对权重矩阵 $W$ 的梯度 $\frac{\partial L_i}{\partial W}$，通过梯度下降来更新权重。

#### **(a) 错误类别的梯度**

对于错误类别 $j \neq y[i]$，当 $s_j - s_{y[i]} + 1 > 0$ 时，损失的梯度为：

$$
\frac{\partial L_i}{\partial W_j} = \mathbf{x}_i
$$

这是因为损失函数对得分 $s_j$ 的梯度是 1，当损失为正时，我们需要增加错误类别的得分。

#### **(b) 正确类别的梯度**

对于正确类别 $j = y[i]$，我们希望 **增大正确类别的得分**，从而减小损失。由于损失函数对于正确类别的得分是 **负的**（即希望该类别的得分增加），所以：

$$
\frac{\partial L_i}{\partial W_{y[i]}} = - \sum_{j \neq y[i]} \frac{\partial L_i}{\partial W_j}
$$

这个操作是为了调整 **正确类别的得分**，通过增加它来减少损失。

### **3. 如何得到 $\mathbf{x}_i$ 作为梯度**

在 SVM 中，**得分** $s_j$ 由输入特征 $x_i$ 和权重 $W_j$ 的内积计算得到：

$$
s_j = W^T x_i
$$

因此，损失函数 $L_i$ 对权重 $W_j$ 的梯度是基于 **得分对权重的变化**。通过链式法则，得出对权重 $W_j$ 的梯度时，梯度包含了输入特征 $x_i$。

#### **计算梯度时，得到 $\mathbf{x}_i$**

具体地，计算损失函数对权重矩阵 $W_j$ 的梯度时，对于每个类别 $j$，我们有：

$$
\frac{\partial L_i}{\partial W_j} = \frac{\partial L_i}{\partial s_j} \cdot \frac{\partial s_j}{\partial W_j}
$$

由于 $s_j = W^T x_i$，我们知道 $\frac{\partial s_j}{\partial W_j} = x_i$，因此梯度公式为：

- **对于错误类别 $j \neq y[i]$**，梯度是 $\mathbf{x}_i$，表示我们需要 **增加错误类别的得分**。

  $$
  \frac{\partial L_i}{\partial W_j} = \mathbf{x}_i
  $$

- **对于正确类别 $j = y[i]$**，梯度是负的，因为我们希望 **减少正确类别的得分**。

  $$
  \frac{\partial L_i}{\partial W_{y[i]}} = -\mathbf{x}_i
  $$

### **4. 总结**

我们通过 **SVM 损失函数** 的定义和 **梯度下降法** 推导出以下梯度计算规则：

- **对于正确类别** $j = y[i]$，梯度是负的 $-\mathbf{x}_i$，因为我们希望 **增大正确类别的得分**，减少损失。
- **对于错误类别** $j \neq y[i]$，梯度是正的 $\mathbf{x}_i$，因为我们希望 **减小错误类别的得分**，从而增大正确类别的得分。

这个负号是基于 SVM 损失函数的优化目标和梯度计算规则的自然结果。
